In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
from collections import Counter

classes = ['Aorta', 'Flows', 'Other', 'V sign', 'X sign']

import os
import cv2
import numpy as np
from torch.utils.data import Dataset

class FetalEchoDataset(Dataset):
    def __init__(self, root_dir, phase='train', transform=None, selected_classes=None, x_sign_mode='both'):
        self.root_dir = root_dir
        self.phase = phase
        self.transform = transform

        all_classes = ['Aorta', 'Flows', 'Other', 'V sign', 'X sign']
        self.classes = selected_classes if selected_classes is not None else all_classes
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.samples = []

        if phase in ['train']:
            for cls in self.classes:
                if cls == 'X sign':
                    # Tùy chọn cách lấy ảnh của X sign
                    if x_sign_mode == 'clear':
                        subfolder = 'X sign/clear'
                        folder_path = os.path.join(root_dir, phase, subfolder)
                        if os.path.exists(folder_path):
                            for fname in os.listdir(folder_path):
                                if fname.endswith('.jpg') or fname.endswith('.png'):
                                    full_path = os.path.join(folder_path, fname)
                                    self.samples.append((full_path, self.class_to_idx['X sign']))
                    elif x_sign_mode == 'deformed':
                        subfolder = 'X sign/deformed'
                        folder_path = os.path.join(root_dir, phase, subfolder)
                        if os.path.exists(folder_path):
                            for fname in os.listdir(folder_path):
                                if fname.endswith('.jpg') or fname.endswith('.png'):
                                    full_path = os.path.join(folder_path, fname)
                                    self.samples.append((full_path, self.class_to_idx['X sign']))
                    elif x_sign_mode == 'both':
                        for subfolder in ['both']:
                            folder_path = os.path.join(root_dir, phase, 'X sign', subfolder)
                            if os.path.exists(folder_path):
                                for fname in os.listdir(folder_path):
                                    if fname.endswith('.jpg') or fname.endswith('.png'):
                                        full_path = os.path.join(folder_path, fname)
                                        self.samples.append((full_path, self.class_to_idx['X sign']))
                else:
                    folder_path = os.path.join(root_dir, phase, cls)
                    if os.path.exists(folder_path):
                        for fname in os.listdir(folder_path):
                            if fname.endswith('.jpg') or fname.endswith('.png'):
                                full_path = os.path.join(folder_path, fname)
                                self.samples.append((full_path, self.class_to_idx[cls]))
        elif phase in ['valid', 'test']:
            for cls in self.classes:
              folder_path = os.path.join(root_dir, phase, cls)
              if os.path.exists(folder_path):
                  for fname in os.listdir(folder_path):
                      if fname.endswith('.jpg') or fname.endswith('.png'):
                          full_path = os.path.join(folder_path, fname)
                          self.samples.append((full_path, self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            img = self.transform(img)
        return img, label

In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
main_folder = '/content/drive/MyDrive/Colab Notebooks/dataset/processedfetal_delation'
# Set image size if needed (optional)
IMG_SIZE = (224, 224)

# Data augmentations for training set
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5),  # Flip ngang
    transforms.RandomApply([
        transforms.RandomRotation(degrees=10)  # Xoay ±10 độ
    ], p=0.75),
    transforms.RandomResizedCrop(size=IMG_SIZE, scale=(1.0, 1.1)),
    # Phóng to trong khoảng 1.0–1.1x và resize về IMG_SIZE
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Thay đổi sáng/tương phản
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),  # Warp đối xứng
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Chuẩn hóa ảnh màu RGB
])
# For validation/test: no augmentation
val_test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Chuẩn hóa ảnh màu RGB
])

# Chỉ dùng X sign rõ cấu trúc
train_dataset_clear = FetalEchoDataset(main_folder, phase='train', transform=train_transforms, x_sign_mode='clear')

# Hoặc chỉ dùng X sign biến dạng
train_dataset_deformed = FetalEchoDataset(main_folder, phase='train', transform=train_transforms, x_sign_mode='deformed')

# Hoặc dùng cả hai (mặc định)
train_dataset_both = FetalEchoDataset(main_folder, phase='train', transform=train_transforms, x_sign_mode='both')


# Load datasets
valid_dataset = FetalEchoDataset(main_folder, phase='valid', transform=val_test_transforms)
test_dataset  = FetalEchoDataset(main_folder, phase='test',  transform=val_test_transforms)

# DataLoaders
BATCH_SIZE = 32
train_dataset_both_loader = DataLoader(train_dataset_both, batch_size=BATCH_SIZE, shuffle=True)
train_dataset_deformed_loader =DataLoader(train_dataset_deformed, batch_size=BATCH_SIZE, shuffle=True)
train_dataset_clear_loader = DataLoader(train_dataset_clear, batch_size=BATCH_SIZE, shuffle=True)

valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import OneCycleLR

# Device & model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.densenet201(num_classes=len(classes)).to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer & scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 30
steps_per_epoch = len(train_dataset_both_loader)
scheduler = OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=steps_per_epoch, epochs=num_epochs)

# Tracking
train_losses, valid_losses, train_accuracies, valid_accuracies = [], [], [], []
best_f1 = 0.0
pathbestmodel = ''
count = 0

for epoch in range(num_epochs):
    model.train()
    running_loss, correct_train, total_train = 0.0, 0, 0

    for inputs, labels in train_dataset_both_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    epoch_loss = running_loss / len(train_dataset_both_loader.dataset)
    train_acc = correct_train / total_train
    train_losses.append(epoch_loss)
    train_accuracies.append(train_acc)

    # Validation
    model.eval()
    valid_loss, correct_val = 0.0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    valid_epoch_loss = valid_loss / len(valid_loader.dataset)
    val_acc = correct_val / len(valid_loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')

    valid_losses.append(valid_epoch_loss)
    valid_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}: "
          f"Train Loss = {epoch_loss:.4f}, Train Acc = {train_acc:.4f}, "
          f"Val Loss = {valid_epoch_loss:.4f}, Val Acc = {val_acc:.4f}, "
          f"F1 Score (macro) = {f1:.4f}")

    # Save best model
    if f1 > best_f1:
        best_f1 = f1
        pathbestmodel = f'/content/drive/MyDrive/Colab Notebooks/model/model_test/DenseNet201_bestweight_{count}.pt'
        torch.save(model.state_dict(), pathbestmodel)
        count += 1
        print("✅ Saved best model!")

Epoch 1: Train Loss = 0.9001, Train Acc = 0.6776, Val Loss = 0.5757, Val Acc = 0.8025, F1 Score (macro) = 0.6229
✅ Saved best model!
Epoch 2: Train Loss = 0.4300, Train Acc = 0.8516, Val Loss = 0.5086, Val Acc = 0.8314, F1 Score (macro) = 0.7313
✅ Saved best model!
Epoch 3: Train Loss = 0.3139, Train Acc = 0.8877, Val Loss = 0.8022, Val Acc = 0.7996, F1 Score (macro) = 0.6704
Epoch 4: Train Loss = 0.2868, Train Acc = 0.8995, Val Loss = 0.4834, Val Acc = 0.8661, F1 Score (macro) = 0.7856
✅ Saved best model!
Epoch 5: Train Loss = 0.3061, Train Acc = 0.8962, Val Loss = 0.4841, Val Acc = 0.8432, F1 Score (macro) = 0.7092
Epoch 6: Train Loss = 0.2930, Train Acc = 0.9020, Val Loss = 0.4545, Val Acc = 0.8521, F1 Score (macro) = 0.7602
Epoch 7: Train Loss = 0.2508, Train Acc = 0.9110, Val Loss = 0.5261, Val Acc = 0.8506, F1 Score (macro) = 0.7523
Epoch 8: Train Loss = 0.2491, Train Acc = 0.9173, Val Loss = 0.4298, Val Acc = 0.8743, F1 Score (macro) = 0.8057
✅ Saved best model!
Epoch 9: Train L

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes))
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

Classification Report:
              precision    recall  f1-score   support

       Aorta       0.71      0.96      0.82        96
       Flows       0.99      0.99      0.99       175
       Other       0.97      0.95      0.96       628
      V sign       0.98      0.93      0.95       362
      X sign       0.74      0.70      0.72        91

    accuracy                           0.93      1352
   macro avg       0.88      0.91      0.89      1352
weighted avg       0.94      0.93      0.94      1352

Confusion Matrix:
[[ 92   0   1   0   3]
 [  0 173   2   0   0]
 [ 20   1 597   6   4]
 [  0   0   9 337  16]
 [ 17   0   8   2  64]]


In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix

# Giả sử bạn đã có:
# - mô hình: model
# - tập test: test_loader
# - thiết bị: device (CPU hoặc GPU)

model.eval()  # chế độ đánh giá
y_true = []
y_pred = []

with torch.no_grad():  # không cần gradient
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# ✅ In kết quả đánh giá

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=classes))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))



Classification Report:
              precision    recall  f1-score   support

       Aorta       0.86      0.86      0.86       133
       Flows       1.00      0.98      0.99       219
       Other       0.96      0.97      0.96       689
      V sign       0.96      0.90      0.93       259
      X sign       0.71      0.87      0.78        79

    accuracy                           0.94      1379
   macro avg       0.90      0.92      0.91      1379
weighted avg       0.94      0.94      0.94      1379

Confusion Matrix:
[[115   0  12   0   6]
 [  0 215   4   0   0]
 [ 11   0 665   4   9]
 [  7   0   7 232  13]
 [  0   0   4   6  69]]
